<a href="https://colab.research.google.com/github/agu3ro/Pytorch-DL/blob/main/07_01_Intro_to_SuperGradients_%2B_Image_Classification_Starter_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 👋🏽 What's up! It's [Harpreet](https://twitter.com/DataScienceHarp)

I'll be guiding you through this notebook. At any point, if you get stuck or have questions, there are three ways to get in touch:

1) Send me an email with your issue: harpreet.sahota@deci.ai

2) Hop into the [Deep Learning Daily (powered by Deci) Discord server](https://discord.gg/p9ecgRhDR8), and let me know what your question is.

3) [Open an issue on GitHub](https://github.com/Deci-AI/super-gradients/issues/new/choose)

---

This tutorial introduces [SuperGradients](https://github.com/Deci-AI/super-gradients) and how to use pre-trained models for image classification. Pre-trained models offer excellent performance with minimal effort, as they have already learned visual features from large datasets.

You'll use a model pretrained on ImageNet from the SuperGradients model zoo and fine-tune it on the MiniPlaces dataset.

This is a barebones introduction to using SuperGradients. It should get you familiar with the core concepts. If you need more details on SuperGradients you can [read the docs](https://www.supergradients.com/).


# 👨🏽‍🔧 Setting up the Environment
You also need to install PyTorch and torchvision, which are the libraries used in this tutorial.

You can install them using `pip`:

In [1]:
import torch
import torchvision
from torchvision import transforms, datasets

# ⬇️ Download and Prepare the Dataset

You'll use the [MiniPlaces dataset](https://github.com/CSAILVision/miniplaces) for this tutorial.

In typical image classification datasets, the folder structure is usually organized in a way that each class of images is stored in its own sub-folder.

This is commonly referred to as the "Image Folder" structure.

When you've downloaded and extracted the MiniPlaces dataset, you'll find the images organized into folders according to their respective classes.

Image folder format looks like this:

```
miniplaces/
|-- train/
|   |-- class1/
|   |   |-- image1.jpg
|   |   |-- image2.jpg
|   |   |-- ...
|   |-- class2/
|   |   |-- image1.jpg
|   |   |-- image2.jpg
|   |   |-- ...
|   |-- ...
|-- val/
|   |-- class1/
|   |   |-- image1.jpg
|   |   |-- image2.jpg
|   |   |-- ...
|   |-- class2/
|   |   |-- image1.jpg
|   |   |-- image2.jpg
|   |   |-- ...
|   |-- ...
```

### 🪜 The following code performs several steps:

1. **Download and Extract Dataset:** Downloads the MiniPlaces dataset and extracts it to the 'miniplaces' directory.

2. **Setup Paths:** Sets up paths for the main dataset, the validation set, and creates a new directory for the test set.

3. **List Classes:** Lists all the classes in the dataset by iterating over the directories in the validation set.

4. **Split Validation Set:** For each class, splits the images in the validation set into a smaller validation set and a new test set using a 60/40 split.

5. **Move Images:** Moves the images for the test set from the validation directory to the test directory, maintaining the original directory structure.

In [2]:
%%capture
!wget https://dissect.csail.mit.edu/datasets/miniplaces.zip --no-check-certificate && unzip miniplaces.zip

In [3]:
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

# set the dataset path
dataset_path = Path('miniplaces')
val_path = dataset_path / 'val'

# get the list of classes
classes = [d.name for d in val_path.iterdir() if d.is_dir()]

# create a test directory
test_path = dataset_path / 'test'
test_path.mkdir(exist_ok=True)

# iterate over each class
for class_name in classes:
    class_dir = val_path / class_name
    class_files = [f.name for f in class_dir.iterdir() if f.is_file()]

    # split the validation set
    val_files, test_files = train_test_split(class_files, test_size=0.4, random_state=42)

    # create new test class directory
    test_class_dir = test_path / class_name
    test_class_dir.mkdir(exist_ok=True)

    # move test files from validation folder to test folder
    for test_file in test_files:
        source = class_dir / test_file
        destination = test_class_dir / test_file
        shutil.move(source, destination)

### 💾 Dataset and Dataloader
In PyTorch, datasets and dataloaders are essential components for efficiently working with large-scale datasets during the training process.

1) **Dataset**: A Dataset in PyTorch represents a collection of data. It could be in-memory data like a list of Python objects or out-of-memory data like images on a disk. The Dataset class provides a standardized way to access this data. You typically subclass the Dataset class and implement the `__getitem__` and `__len__` methods to return a single data item and the size of the dataset, respectively. PyTorch also provides several pre-defined Dataset classes like ImageFolder for common use-cases.

2) **DataLoader**: While the Dataset class provides a way to access individual data items, the DataLoader class provides a way to iterate over a Dataset in batches. This is important because when training a neural network, you typically don't feed in the entire dataset at once but process it in smaller batches. The DataLoader also makes it easy to shuffle the data and use multiple subprocesses to load the data in parallel.

To create datasets and dataloaders for the MiniPlaces dataset, you can use PyTorch's `ImageFolder` and `DataLoader` classes.

The `ImageFolder` class allows you to load the images directly from the directory structure, and the `DataLoader` class provides an efficient way to iterate over these images during training.


In [4]:
import torch
from torchvision import datasets, transforms

# define the transform
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # resize images to 224x224
    transforms.ToTensor(),  # convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalize images to ImageNet mean and sd
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # resize images to 224x224
    transforms.ToTensor(),  # convert image to PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # normalize images to ImageNet mean and sd
])

# define the datasets
train_dataset = datasets.ImageFolder(dataset_path / 'train', transform=train_transform)
val_dataset = datasets.ImageFolder(dataset_path / 'val', transform=test_transform)
test_dataset = datasets.ImageFolder(dataset_path / 'test', transform=test_transform)

# define the dataloaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

<img src='https://raw.githubusercontent.com/Deci-AI/super-gradients/master/documentation/assets/SG_img/SG%20-%20Horizontal%20Glow%202.png'>

# 🦸🏾‍♂️ SuperGradients

SuperGradients is a PyTorch based training library.

It provides a uniform interface for the most common computer vision use cases:

- Classification

- Detection

- Segmentation

- Pose estimation

There are nearly 40 pretrained models in the model zoo. You can see the pretrained models available to you by following [this link](https://github.com/Deci-AI/super-gradients/blob/master/documentation/source/model_zoo.md).

This notebook will focus on using SuperGradients for image classification.

In [ ]:
%%capture
!pip install super-gradients==3.2.0 --force-reinstall

# 🪡 Fine-tuning on a custom dataset

## 🏋🏽 The trainer

The first thing you need to define in SuperGradients is the Trainer.

The trainer is in charge of training, evaluation, saving checkpoints, etc. If you're interested in seeing the source code for the trainer, you can do so [here](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/training/sg_trainer/sg_trainer.py).

### ✌🏼 There's two important arguments to the trainer:

1) `ckpt_root_dir` - this is the directory where results from all your experiments will be saved

2)`experiment_name` - all checkpoints, logs, and tensorboards will be saved in a directory with the name you specify here.

SuperGradients supports **Data Parallel** and **Distributed Data Parallel**.

That's outside of the scope for this introduction to SuperGradients.

But, if you're fortunate enough to have multiple GPUs at your disposal or want learn more you can do so [here](https://github.com/Deci-AI/super-gradients/blob/0fe46cd39572db34eb83d68e343fed97b8886fe9/documentation/source/device.md#3-dp---data-parallel).

In the code below, you'll instantiate the trainer with just a single GPU (since that's what Google Colab provides)

In [16]:
%%capture
!pip install --upgrade super-gradients torch torchvision

In [5]:
%%capture
from super_gradients.training import Trainer
CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='hello_world', ckpt_root_dir=CHECKPOINT_DIR)

ModuleNotFoundError: No module named 'super_gradients'

# 👩🏽‍🦳 Instantiating the model

Below is how to instantiate the model for finetuning. Note you need to add the `num_classes` argument here. SG will automatically replace the classification head to match the number of classes you're working with


In [11]:
%%capture
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, 100)
#model = models.get('resnet18', num_classes=100, pretrained_weights='imagenet') doesnt work for some reason its a shame

# 🎛️ Training parameters

You need to define the training parameters for your training run.

Full details about the training parameters can be found [here](https://github.com/Deci-AI/super-gradients/blob/master/src/super_gradients/recipes/training_hyperparams/default_train_params.yaml).

# 🚨 There are a few **mandatory** arguments that you must define for training params 🚨

- `initial_lr` - The initial learning rate for training

The learning rate is a hyperparameter that determines the step size at which the model's weights are updated during training.

It controls how much the model's weights should be adjusted with respect to the loss gradient.

A high learning rate may cause the training to converge quickly, but it can overshoot the optimal solution or cause instability.

A low learning rate may lead to more precise convergence to the optimal solution but can be very slow and possibly get stuck in local minima.

It balances the speed of convergence and the stability of learning, and it often requires careful tuning to find the optimal value for a given problem.

- `max_epochs` - Max number of training epochs

An epoch refers to one complete pass through the entire training dataset.

During an epoch, the model's weights are updated as it processes each batch of samples in the training dataset.

Once every sample has been seen and used to update the weights, one epoch is completed.

- `loss` - the loss function you want to use

- `optimizer` - Optimizer you will be using

- `train_metrics_list` - Metrics to log during training

- `valid_metrics_list` - Metrics to log during validation

- `metric_to_watch` - metric which the model checkpoint will be saved according to

You can choose from a variety of `optimizer`'s such as: Adam, AdamW, SGD, Lion, or RMSProps. If you choose to change the defualt parameters of these optimizrs you pass them into `optimizer_params`.

### Train/val metrics definitions:

**Accuracy** -  known as Top-1 accuracy, this metric tells you the ratio of correct predictions to the total number of items.  It answers the question, "How often does the model predict the exact correct class?"

**Top-5** - If one of the model’s top 5 highest probability predictions match the ground truth. If it does, you count it as a correct prediction. It answers the question, "How often is the correct class within the top 5 guesses of the model?"

Accuracy is often used for tasks where the exact prediction is crucial. Top-5 Accuracy is more common in scenario where there may be ambiguity, and being close to the correct answer is still valuable.

# Optional, but good to know parameters

### `optimizer_params` - The parameters of the optimizer you're using, leaving it blank will use defaults

[For example, there are the parameters for Adam](https://pytorch.org/docs/stable/generated/torch.optim.Adam.html#torch.optim.Adam)


### `criterion_params` - Activates label smoothing cross entropy

In the regular cross entropy loss, we're very strict about our labels: the correct class has a value of 1, and all other classes have a value of 0.

But this can sometimes lead to overfitting, where the model becomes too confident about its predictions and performs poorly on unseen data.

Label smoothing cross-entropy builds upon regular cross-entropy by adding a touch of "softness" to the labels, leading to a model that's confident but not excessively so.

**Label Smoothing Cross-Entropy**: Instead of using strict 0s and 1s in the true labels, label smoothing uses slightly "softer" values (e.g., [0.9, 0.05, 0.05] for class 1).

Improvement:
 - **Avoiding Overconfidence:** It prevents the model from becoming overly confident about its predictions. It acts as a form of regularization that can improve generalization to unseen data.

 - **Stability in Training:** Makes the training process more stable, especially when dealing with noisy or incorrect labels.

Label smoothing is a simple yet effective technique to make the training process more robust and prevent overfitting.

By softening the labels, we encourage the model to be less confident in its predictions, which can lead to better performance on unseen data. It's like telling the model, "Be confident, but not too confident!"

Smoothing Factor (epsilson): This is a value between 0 and 1 that determines the degree of smoothing applied to the labels.

A smoothing factor of 0 means no smoothing (i.e., traditional cross-entropy), while a value closer to 1 means more smoothing.

The smoothing factor is a hyperparameter that you would typically choose based on validation performance or through hyperparameter tuning. It allows you to control how "soft" the labels are, providing a way to regulate the model's confidence in its predictions.


### Learning rate scheduling parameters: `warmup_mode`, `lr_warmup_epochs`, `lr_warmup_steps`, `lr_cooldown_epochs` `warmup_initial_lr`

Learning rate scheduling involves adjusting the learning rate during training. Instead of using a constant learning rate, the learning rate is changed according to a predefined schedule or rule.

It helps in finding a balance between fast convergence (using a high learning rate) and stability (using a low learning rate). Common methods include step decay, exponential decay, and cosine annealing.

- Warm-Up: Starts with a small learning rate and gradually increases it at the beginning of training to avoid large, unstable updates.

- Cool-Down: Gradually reduces the learning rate towards the end of training to help the model settle into a minimum.

Learning rate scheduling, including warm-up and cool-down, is generally beneficial in the following scenarios:

1. Large Datasets: These techniques help the model converge more efficiently.

2. Deep Networks: For deep networks with many layers, controlling the learning rate prevents overshooting and helps the model reach a better minimum.

3. Avoiding Overfitting: Gradually reducing the learning rate (cool-down) helps in fine-tuning the model and reduces the risk of overfitting.

4. Training Stability: Warm-up can prevent large updates at the beginning of training, leading to a more stable training process.

5. Exploration vs Exploitation: By controlling the learning rate, you can balance the exploration of the loss landscape (with higher learning rates) and exploitation of promising areas (with lower learning rates).

Together, these techniques guide the learning process, aiding in efficient and robust convergence of the model.

In [7]:
train_params = {
    "initial_lr": 3e-1,
    "max_epochs":10,
    "loss":"cross_entropy",
    "optimizer":"Adam", # you can choose from Adam, AdamW, SGD, Lion, or RMSProps
    "train_metrics_list":["Accuracy", "Top5"],
    "valid_metrics_list":["Accuracy", "Top5"],
    "metric_to_watch":"Accuracy",  # will be the metric which the model checkpoint will be saved according to

    # optional parameters
    "optimizer_params": {}, # when `optimizer` is one of ['Adam','SGD','RMSProp'], it will be initialized with optimizer_params.
    "criterion_params": {'smooth_eps': 0.1}, # Enable label-smoothing cross-entropy
    "warmup_initial_lr": 3e-4, # Initial lr for linear_epoch_step/linear_batch_step. When none is given, initial_lr/(warmup_epochs+1) will be used.
    "warmup_mode": "linear_epoch_step", # learning rate warmup scheme, currently 'linear_epoch_step' and 'linear_batch_step' are supported
    "lr_warmup_epochs": 3, # number of epochs for learning rate warm up
    "lr_warmup_steps": 2,  # number of warmup steps
    "lr_cooldown_epochs": 1 # epochs to cooldown LR (i.e the last epoch from scheduling view point=max_epochs-cooldown)
}

In [8]:
train_params

{'initial_lr': 0.3,
 'max_epochs': 10,
 'loss': 'cross_entropy',
 'optimizer': 'Adam',
 'train_metrics_list': ['Accuracy', 'Top5'],
 'valid_metrics_list': ['Accuracy', 'Top5'],
 'metric_to_watch': 'Accuracy',
 'optimizer_params': {},
 'criterion_params': {'smooth_eps': 0.1},
 'warmup_initial_lr': 0.0003,
 'warmup_mode': 'linear_epoch_step',
 'lr_warmup_epochs': 3,
 'lr_warmup_steps': 2,
 'lr_cooldown_epochs': 1}


### 🪄 SuperGradients offers a number of training tricks right out of the box, such as:

- Exponential moving average
- Zero weight decay on bias and batch normalizatiom
- Weight averaging
- Batch accumulation
- Precise BatchNorm

You can read more details about these training tricks [here](https://heartbeat.comet.ml/a-better-way-to-train-your-neural-networks-813b60a5bd6a).

If you're interested in building a using a custom metric with SuperGradients you can learn how [here](https://github.com/Deci-AI/super-gradients/blob/master/documentation/source/Metrics.md).


# 🦾 Training the model

You've covered a lot of ground so far:

✅ Instantiated the trainer

✅ Defined your dataset parameters and dataloaders

✅ Instantiated a model

✅ Set up your training parameters

### ⏳ Now, its time to train a model

Training a model using a SuperGradients is done using the `trainer`.

It's as easy as...

In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_loader,
              valid_loader=val_loader)

# 🏆 Get the best trained model

Now that training is complete, you need to get the best trained model.

You used checkpoint averaging so the following code will use weights averaged across training runs.

If you want to use the best weights, or weights from the last epoch you'd use one of the following in the code below:

- best weights: `checkpoint_path = content/checkpoints/hello_world/ckpt_best.pth`

- last weights: `checkpoint_path = content/checkpoints/hello_world/ckpt_latest.pth`

- average weights: `checkpoint_path = content/checkpoints/hello_world/average_model.pth`

In [14]:
%%capture
!pip install super-gradients==3.2.0

In [15]:
from super_gradients.training import models
best_model = models.get('resnet18',
                        num_classes=len(train_dataset.classes),
                        checkpoint_path="/content/checkpoints/hello_world/best_model.pth")

[2024-11-04 18:48:50] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


TypeError: issubclass() arg 1 must be a class

# 🧐 Evaluating the best trained model on the test set

In [ ]:
trainer.test(model=best_model,test_loader=test_loader)

# 🔮 Predicting with the best model

The next line will perform detection on the following image. Note, we didn't have a class for the half dollar coin. So it will likely get classified as something else.






In [ ]:
from super_gradients.training.processing.processing import default_imagenet_processing_params
processing_params = default_imagenet_processing_params()
processing_params['class_names'] = train_dataset.classes
best_model.set_dataset_processing_params(**processing_params)

In [ ]:
img_url = '/content/miniplaces/test/aquarium/00000248.jpg'
best_model.predict(img_url).show()

# 💫 I think you're ready to venture out on your own now!

I've created a templated notebook for you [here](https://colab.research.google.com/drive/1HWU6oOEVfh3sS59KHH3q6StG1ZtO1D2Q).

If you run into any issues, you know how to get a hold of me (contact info is at the top of the notebook).

Cheers and I can't wait to see what you come up with!